In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt
import statistics 
from scipy.stats import *
from scipy.spatial import distance

######### Instance Transfer repositories ####################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Repositories uploaded!!")

Repositories uploaded!!


In [7]:
################################################################ Ailerons ###############################################
target_ailerons = ['goal']
colnames_ailerons = ['climbRate', 'Sgz', 'p', 'q', 'curPitch', 'curRoll', 'absRoll', 'diffClb', 'diffRollRate', 'diffDiffClb', 'SeTime1',
            'SeTime2', 'SeTime3', 'SeTime4', 'SeTime5', 'SeTime6', 'SeTime7', 'SeTime8', 'SeTime9', 'SeTime10', 'SeTime11', 'SeTime12', 
            'SeTime13', 'SeTime14', 'diffSeTime1', 'diffSeTime2', 'diffSeTime3', 'diffSeTime4', 'diffSeTime5', 'diffSeTime6', 'diffSeTime7',
            'diffSeTime8', 'diffSeTime9', 'diffSeTime10', 'diffSeTime11', 'diffSeTime12', 'diffSeTime13', 'diffSeTime14', 'alpha', 'Se', 'goal']
AileronsData_train_df = pd.read_csv('UCI_regression/Ailerons/ailerons.data', header = None, names = colnames_ailerons) 
print("Ailerons Data")
print("Training Set: ", AileronsData_train_df.shape)

AileronsData_test_df = pd.read_csv('UCI_regression/Ailerons/ailerons.test', header = None, names = colnames_ailerons) 
print("Testing Set: ", AileronsData_test_df.shape)

print("-------------------------------------------")

############################################### Standardization ###############################################
ailerons_cols = AileronsData_train_df.columns.difference(['goal'])
ss = StandardScaler()
AileronsData_train_df[ailerons_cols] = ss.fit_transform(AileronsData_train_df[ailerons_cols])
AileronsData_test_df[ailerons_cols] = ss.transform(AileronsData_test_df[ailerons_cols]) ## Use the same scale as the training data


#################### Splitting with small target set and large source and test set #############
AileronsData_source_df, AileronsData_tgt_df = train_test_split(AileronsData_train_df, test_size = 0.05) ## test_size = tgt size
# print(AileronsData_df_tgt.shape, AileronsData_df_source.shape, AileronsData_df_test.shape)

AileronsData_train_df = AileronsData_train_df.reset_index(drop = True)
AileronsData_tgt_df = AileronsData_tgt_df.reset_index(drop = True)
AileronsData_source_df = AileronsData_source_df.reset_index(drop = True)
print("Target Set: ", AileronsData_tgt_df.shape)
print("Source Set: ", AileronsData_source_df.shape)
print("Test Set: ", AileronsData_test_df.shape)


# ################################# Splitting into predictors and target data #################################
# AileronsData_df_test_y = AileronsData_test_df[target_ailerons]
# AileronsData_df_test_X = AileronsData_test_df.drop(target_ailerons, axis = 1)

# AileronsData_df_tgt_y = AileronsData_tgt_df[target_ailerons]
# AileronsData_df_tgt_X = AileronsData_tgt_df.drop(target_ailerons, axis = 1)

# AileronsData_df_source_y = AileronsData_source_df[target_ailerons]
# AileronsData_df_source_X = AileronsData_source_df.drop(target_ailerons, axis = 1)


# ############## Merging the datasets #################
# ailerons_X_df = pd.concat([AileronsData_df_source_X, AileronsData_df_tgt_X], ignore_index=True)
# ailerons_y_df = pd.concat([AileronsData_df_source_y, AileronsData_df_tgt_y], ignore_index=True)

# ailerons_np_train_X = ailerons_X_df.to_numpy()
# ailerons_np_train_y = ailerons_y_df.to_numpy()

# ailerons_np_test_X = AileronsData_df_test_X.to_numpy()
# ailerons_np_test_y = AileronsData_df_test_y.to_numpy()

# ailerons_np_train_y_list = ailerons_np_train_y.ravel()
# ailerons_np_test_y_list = ailerons_np_test_y.ravel()

# src_size_ailerons = len(AileronsData_df_source_y)
# tgt_size_ailerons = len(AileronsData_df_tgt_y)

# print("-------------------------------------------")
###############################################################################################################################################

Ailerons Data
Training Set:  (7154, 41)
Testing Set:  (6596, 41)
-------------------------------------------
Target Set:  (358, 41)
Source Set:  (6796, 41)
Test Set:  (6596, 41)


In [8]:
######################################## Variance Sampling: Phase 1###################################################
kmeans = KMeans(n_clusters = 15, random_state=0).fit(AileronsData_tgt_df)

AileronsData_alternate_df = AileronsData_tgt_df.copy()
AileronsData_alternate_df_np = AileronsData_tgt_df.to_numpy()

idxlist = []
AileronsData_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in AileronsData_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", AileronsData_alternate_df_np.shape)
    AileronsData_new_df_list.append(rowval)
    AileronsData_alternate_df = np.delete(AileronsData_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


AileronsData_new_df = pd.DataFrame(np.vstack(AileronsData_new_df_list))
AileronsData_new_df.shape

(15, 41)

In [9]:
##################################################### Variance Sampling: Phase 2 ################################################

AileronsData_alternate_source_df = AileronsData_source_df.copy()
AileronsData_alternate_source_df_np = AileronsData_alternate_source_df.to_numpy()

idxlist2 = []
AileronsData_final_df_list = []

for row_nm in AileronsData_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in AileronsData_alternate_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", AileronsData_alternate_source_df_np.shape)
    AileronsData_final_df_list.append(row_val)
    AileronsData_alternate_source_df_np = np.delete(AileronsData_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


AileronsData_final_df = pd.DataFrame(np.vstack(AileronsData_final_df_list), columns= AileronsData_alternate_source_df.columns)


print("Shape of source dataset before Variance Sampling: ",AileronsData_source_df.shape)
AileronsData_source_df = pd.DataFrame(np.vstack(AileronsData_alternate_source_df_np), columns= AileronsData_source_df.columns)
print("Shape of source dataset after Variance Sampling: ", AileronsData_source_df.shape)

print("----------------------------------------------")

print("Shape of target before Variance Sampling: ", AileronsData_tgt_df.shape)
AileronsData_tgt_df = pd.concat([AileronsData_tgt_df, AileronsData_final_df], ignore_index=True)
print("Shape of target after Variance Sampling: ", AileronsData_tgt_df.shape)

print("----------------------------------------------")

Shape of source dataset before Variance Sampling:  (6796, 41)
Shape of source dataset after Variance Sampling:  (6781, 41)
----------------------------------------------
Shape of target before Variance Sampling:  (358, 41)
Shape of target after Variance Sampling:  (373, 41)
----------------------------------------------


In [10]:
################################## Importance Sampling: Manhattan Distance ######################################################

AileronsData_source_df["ManDis"] = ""

AileronsData_tgt_df_mean = []
prow = AileronsData_tgt_df.mean()
AileronsData_tgt_df_mean = [prow.climbRate, prow.Sgz, prow.p, prow.q, prow.curPitch, prow.curRoll, prow.absRoll,
       prow.diffClb, prow.diffRollRate, prow.diffDiffClb, prow.SeTime1, prow.SeTime2, prow.SeTime3, prow.SeTime4, 
        prow.SeTime5, prow.SeTime6, prow.SeTime7, prow.SeTime8, prow.SeTime9, prow.SeTime10, prow.SeTime11, prow.SeTime12, 
        prow.SeTime13, prow.SeTime14, prow.diffSeTime1, prow.diffSeTime2, prow.diffSeTime3, prow.diffSeTime4, prow.diffSeTime5, 
        prow.diffSeTime6, prow.diffSeTime7, prow.diffSeTime8, prow.diffSeTime9, prow.diffSeTime10, prow.diffSeTime11, prow.diffSeTime12,
       prow.diffSeTime13, prow.diffSeTime14, prow.alpha, prow.Se, prow.goal]

rowidx = 0

for row in AileronsData_source_df.itertuples():
    row_list =[row.climbRate, row.Sgz, row.p, row.q, row.curPitch, row.curRoll, row.absRoll,
       row.diffClb, row.diffRollRate, row.diffDiffClb, row.SeTime1, row.SeTime2, row.SeTime3, row.SeTime4, 
        row.SeTime5, row.SeTime6, row.SeTime7, row.SeTime8, row.SeTime9, row.SeTime10, row.SeTime11, row.SeTime12, 
        row.SeTime13, row.SeTime14, row.diffSeTime1, row.diffSeTime2, row.diffSeTime3, row.diffSeTime4, row.diffSeTime5, 
        row.diffSeTime6, row.diffSeTime7, row.diffSeTime8, row.diffSeTime9, row.diffSeTime10, row.diffSeTime11, row.diffSeTime12,
       row.diffSeTime13, row.diffSeTime14, row.alpha, row.Se, row.goal]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = AileronsData_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    AileronsData_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

AileronsData_source_df = AileronsData_source_df.sort_values(by =['ManDis'])
AileronsData_source_df = AileronsData_source_df.head(1500) 
AileronsData_source_df = AileronsData_source_df.drop(['ManDis'], axis =1)
AileronsData_source_df = AileronsData_source_df.reset_index(drop=True)

# print("Target Set: ",AileronsData_tgt_df.shape)
# print("Source Set: ",AileronsData_source_df.shape)
# print("Test Set: ",AileronsData_test_df.shape)


################################# Splitting into predictors and target data #################################
print("Target Set: ", AileronsData_tgt_df.shape)
print("Source Set: ", AileronsData_source_df.shape)
print("Test Set: ", AileronsData_test_df.shape)


AileronsData_df_test_y = AileronsData_test_df[target_ailerons]
AileronsData_df_test_X = AileronsData_test_df.drop(target_ailerons, axis = 1)

AileronsData_df_tgt_y = AileronsData_tgt_df[target_ailerons]
AileronsData_df_tgt_X = AileronsData_tgt_df.drop(target_ailerons, axis = 1)

AileronsData_df_source_y = AileronsData_source_df[target_ailerons]
AileronsData_df_source_X = AileronsData_source_df.drop(target_ailerons, axis = 1)


############## Merging the datasets #################
ailerons_X_df = pd.concat([AileronsData_df_tgt_X, AileronsData_df_source_X], ignore_index=True)
ailerons_y_df = pd.concat([AileronsData_df_tgt_y, AileronsData_df_source_y], ignore_index=True)

ailerons_np_train_X = ailerons_X_df.to_numpy()
ailerons_np_train_y = ailerons_y_df.to_numpy()

ailerons_np_test_X = AileronsData_df_test_X.to_numpy()
ailerons_np_test_y = AileronsData_df_test_y.to_numpy()

ailerons_np_train_y_list = ailerons_np_train_y.ravel()
ailerons_np_test_y_list = ailerons_np_test_y.ravel()

src_size_ailerons = len(AileronsData_df_source_y)
tgt_size_ailerons = len(AileronsData_df_tgt_y)

print("-------------------------------------------")

###############################################################################################################################################

Target Set:  (373, 41)
Source Set:  (1500, 41)
Test Set:  (6596, 41)
-------------------------------------------


In [ ]:
#################################### STrAdaBoost.R2 Active Sampling Ailerons ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(AileronsData_df_tgt_X), len(AileronsData_df_source_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_ailerons = []
rmselist_stradaboost_ailerons = []

for x in range(0, 10):

    model_stradaboost_ailerons = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)


    y_pred_stradaboost_ailerons = model_stradaboost_ailerons.predict(ailerons_np_test_X)

    mse_stradaboost_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_stradaboost_ailerons))
    rmselist_stradaboost_ailerons.append(mse_stradaboost_ailerons)
        
    r2_score_stradaboost_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_stradaboost_ailerons)
    r2_score_stradaboost_ailerons = (r2_score_stradaboost_ailerons[0])**2
    r2scorelist_stradaboost_ailerons.append(r2_score_stradaboost_ailerons)

print("RMSE of STrAdaboostR2:", rmselist_stradaboost_ailerons)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_ailerons)

print("\n")

print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_ailerons), statistics.stdev(rmselist_stradaboost_ailerons))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_ailerons), statistics.stdev(r2scorelist_stradaboost_ailerons))

print("-------------------------------------------")

In [ ]:
########################### Two-Stage TrAdaBoost.R2 Ailerons #######################################

src_idx_ailerons = np.arange(start = 0, stop = (src_size_ailerons - 1), step = 1)
tgt_idx_ailerons = np.arange(start = src_size_ailerons, stop = ((src_size_ailerons + tgt_size_ailerons)-1), step = 1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}


from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_ailerons = []
rmselist_TwoTrAda_ailerons = []

for x in range(0, 20):

    model_TwoTrAda_ailerons = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv = 10) 
    model_TwoTrAda_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list, src_idx_ailerons, tgt_idx_ailerons)

    y_pred_TwoTrAda_ailerons = model_TwoTrAda_ailerons.predict(ailerons_np_test_X)

    mse_TwoTrAda_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_TwoTrAda_ailerons))
    rmselist_TwoTrAda_ailerons.append(mse_TwoTrAda_ailerons)
        
    r2_score_TwoTrAda_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_TwoTrAda_ailerons)
    r2_score_TwoTrAda_ailerons = (r2_score_TwoTrAda_ailerons[0])**2
    r2scorelist_TwoTrAda_ailerons.append(r2_score_TwoTrAda_ailerons)


print("RMSE of TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_ailerons))
print("R^2 of TrAdaboost.R2:", statistics.mean(r2scorelist_TwoTrAda_ailerons))
print("\n")
print("RMSE of TrAdaboost.R2:", rmselist_TwoTrAda_ailerons)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_ailerons)


print("-------------------------------------------")

In [ ]:
########################### AdaBoostR2 Transfer Learning Ailerons #####################################################
from sklearn.ensemble import AdaBoostRegressor

print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_ailerons = []
rmselist_AdaTL_ailerons = []

for x in range(0, 30):

    model_AdaTL_ailerons = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)

    y_pred_AdaTL_ailerons = model_AdaTL_ailerons.predict(ailerons_np_test_X)

    mse_AdaTL_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_AdaTL_ailerons))
    rmselist_AdaTL_ailerons.append(mse_AdaTL_ailerons)
        
    r2_score_AdaTL_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_AdaTL_ailerons)
    r2_score_AdaTL_ailerons = (r2_score_AdaTL_ailerons[0])**2
    r2scorelist_AdaTL_ailerons.append(r2_score_AdaTL_ailerons)

print("RMSE of Adaboost.R2:", statistics.mean(rmselist_AdaTL_ailerons))
print("R^2 of Adaboost.R2:", statistics.mean(r2scorelist_AdaTL_ailerons))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_AdaTL_ailerons)
print("R^2 of Adaboost.R2:", r2scorelist_AdaTL_ailerons)


print("-------------------------------------------")

In [ ]:
######################### Gradient Boosting Regression Transfer Learning Ailerons #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_ailerons = []
rmselist_GBRTL_ailerons = []

for x in range(0, 20):

    model_GBRTL_ailerons = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)

    y_pred_GBRTL_ailerons = model_GBRTL_ailerons.predict(AileronsData_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_GBRTL_ailerons))
    rmselist_GBRTL_ailerons.append(mse_GBRTL_ailerons)
        
    r2_score_GBRTL_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_GBRTL_ailerons)
    r2_score_GBRTL_ailerons = (r2_score_GBRTL_ailerons[0])**2
    r2scorelist_GBRTL_ailerons.append(r2_score_GBRTL_ailerons)

    
print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_ailerons))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_ailerons))
print("\n")
print("RMSE of GBRTL:", rmselist_GBRTL_ailerons)
print("R^2 of GBRTL:", r2scorelist_GBRTL_ailerons)


print("-------------------------------------------")

In [14]:
####################################### Elevators #########################################################
target_elevators = ['Goal']
colnames_elevators = ['climbRate', 'Sgz', 'p', 'q', 'curRoll', 'absRoll', 'diffClb', 'diffRollRate', 'diffDiffClb', 'SaTime1', 'SaTime2', 
                      'SaTime3', 'SaTime4', 'diffSaTime1', 'diffSaTime2', 'diffSaTime3', 'diffSaTime4', 'Sa', 'Goal']
Elevators_train_df = pd.read_csv('UCI_regression/Elevators/elevators.data', header = None, names = colnames_elevators)
print("Elevators Data")
print(Elevators_train_df.shape)

Elevators_test_df = pd.read_csv('UCI_regression/Elevators/elevators.test', header = None, names = colnames_elevators)
print(Elevators_test_df.shape)

############################################### Standardization ###############################################
elevators_cols = Elevators_train_df.columns.difference(['Goal'])
ss = StandardScaler()
Elevators_train_df[elevators_cols] = ss.fit_transform(Elevators_train_df[elevators_cols])
Elevators_test_df[elevators_cols] = ss.transform(Elevators_test_df[elevators_cols]) ## Use the same scale as the training data

#################### Splitting with small target set and large source and test set #############
Elevators_source_df, Elevators_tgt_df = train_test_split(Elevators_train_df, test_size = 0.05) ## test_size = tgt size
# print(Elevators_df_tgt.shape, Elevators_df_source.shape, Elevators_df_test.shape)

Elevators_tgt_df = Elevators_tgt_df.reset_index(drop = True)
Elevators_source_df = Elevators_source_df.reset_index(drop = True)
print("Target Set: ", Elevators_tgt_df.shape)
print("Source Set: ", Elevators_source_df.shape)
print("Test Set: ", Elevators_test_df.shape)

# ################################# Splitting into predictors and target data #################################
# Elevators_test_df_y = Elevators_test_df[target_elevators]
# Elevators_test_df_X = Elevators_test_df.drop(target_elevators, axis = 1)

# Elevators_tgt_df_y = Elevators_tgt_df[target_elevators]
# Elevators_tgt_df_X = Elevators_tgt_df.drop(target_elevators, axis = 1)

# Elevators_source_df_y = Elevators_source_df[target_elevators]
# Elevators_source_df_X = Elevators_source_df.drop(target_elevators, axis = 1)


# ############## Merging the datasets #################
# elevators_X_df = pd.concat([Elevators_source_df_X, Elevators_tgt_df_X], ignore_index=True)
# elevators_y_df = pd.concat([Elevators_source_df_y, Elevators_tgt_df_y], ignore_index=True)

# elevators_np_train_X = elevators_X_df.to_numpy()
# elevators_np_train_y = elevators_y_df.to_numpy()

# elevators_np_test_X = Elevators_test_df_X.to_numpy()
# elevators_np_test_y = Elevators_test_df_y.to_numpy()

# elevators_np_train_y_list = elevators_np_train_y.ravel()
# elevators_np_test_y_list = elevators_np_test_y.ravel()

# src_size_elevators = len(Elevators_source_df_y)
# tgt_size_elevators = len(Elevators_tgt_df_y)

# print("---------------------------")

# ###############################################################################################################################################

Elevators Data
(8752, 19)
(7847, 19)
Target Set:  (438, 19)
Source Set:  (8314, 19)
Test Set:  (7847, 19)


In [15]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 20, random_state=0).fit(Elevators_tgt_df)

Elevators_alternate_df = Elevators_tgt_df.copy()
Elevators_alternate_df_np = Elevators_tgt_df.to_numpy()

idxlist = []
Elevators_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in Elevators_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", Elevators_alternate_df_np.shape)
    Elevators_new_df_list.append(rowval)
    Elevators_alternate_df = np.delete(Elevators_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


Elevators_new_df = pd.DataFrame(np.vstack(Elevators_new_df_list))
Elevators_new_df.shape

##################################################### Variance Sampling: Phase 2 ################################################

Elevators_alternate_source_df = Elevators_source_df.copy()
Elevators_alternate_source_df_np = Elevators_alternate_source_df.to_numpy()

idxlist2 = []
Elevators_final_df_list = []

for row_nm in Elevators_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in Elevators_alternate_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", Elevators_alternate_source_df_np.shape)
    Elevators_final_df_list.append(row_val)
    Elevators_alternate_source_df_np = np.delete(Elevators_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


Elevators_final_df = pd.DataFrame(np.vstack(Elevators_final_df_list), columns= Elevators_alternate_source_df.columns)

print("Shape of source before :",Elevators_source_df.shape)
Elevators_source_df = pd.DataFrame(np.vstack(Elevators_alternate_source_df_np), columns= Elevators_source_df.columns)
print("Shape of source after :", Elevators_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", Elevators_tgt_df.shape)
Elevators_tgt_df = pd.concat([Elevators_tgt_df, Elevators_final_df], ignore_index=True)
print("Shape of target after :", Elevators_tgt_df.shape)

print("----------------------------------------------")

----------------------------------------------
Shape of source before : (8314, 19)
Shape of source after : (8294, 19)
----------------------------------------------
Shape of target before : (438, 19)
Shape of target after : (458, 19)
----------------------------------------------


In [16]:
################################## Importance Sampling: Manhattan Distance ######################################################

Elevators_source_df["ManDis"] = ""

Elevators_tgt_df_mean = []
prow = Elevators_tgt_df.mean()
Elevators_tgt_df_mean = [prow.climbRate, prow.Sgz, prow.p, prow.q, prow.curRoll, prow.absRoll,
       prow.diffClb, prow.diffRollRate, prow.diffDiffClb, prow.SaTime1, prow.SaTime2, prow.SaTime3, prow.SaTime4, 
        prow.diffSaTime1, prow.diffSaTime2, prow.diffSaTime3, prow.diffSaTime4, prow.Sa, prow.Goal]

rowidx = 0

for row in Elevators_source_df.itertuples():
    row_list =[row.climbRate, row.Sgz, row.p, row.q, row.curRoll, row.absRoll, row.diffClb, 
            row.diffRollRate, row.diffDiffClb, row.SaTime1, row.SaTime2, row.SaTime3, row.SaTime4, 
               row.diffSaTime1, row.diffSaTime2, row.diffSaTime3, row.diffSaTime4, row.Sa, row.Goal]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = Elevators_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    Elevators_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

Elevators_source_df = Elevators_source_df.sort_values(by =['ManDis'])
Elevators_tgt_source_df = Elevators_source_df.head(2078) ## 600 instances chosen for Kinematics Dataset
Elevators_source_df = Elevators_source_df.iloc[2078:]

Elevators_source_df = Elevators_source_df.drop(['ManDis'], axis =1)
Elevators_tgt_source_df = Elevators_tgt_source_df.drop(['ManDis'], axis =1)

Elevators_tgt_df = pd.concat([Elevators_tgt_df, Elevators_tgt_source_df], ignore_index=True) ### This line is used only for STrAdaBoost.R2 and not for TrAdaBoost.R2

Elevators_tgt_df = Elevators_tgt_df.reset_index(drop=True)
Elevators_source_df = Elevators_source_df.reset_index(drop=True)


#################### Splitting with small target set and large source and test set #############
print("Target Set: ", Elevators_tgt_df.shape)
print("Source Set: ", Elevators_source_df.shape)
print("Test Set: ", Elevators_test_df.shape)


Elevators_test_df_y = Elevators_test_df[target_elevators]
Elevators_test_df_X = Elevators_test_df.drop(target_elevators, axis = 1)

Elevators_tgt_df_y = Elevators_tgt_df[target_elevators]
Elevators_tgt_df_X = Elevators_tgt_df.drop(target_elevators, axis = 1)

Elevators_source_df_y = Elevators_source_df[target_elevators]
Elevators_source_df_X = Elevators_source_df.drop(target_elevators, axis = 1)


############## Merging the datasets #################
elevators_X_df = pd.concat([Elevators_tgt_df_X, Elevators_source_df_X], ignore_index=True)
elevators_y_df = pd.concat([Elevators_tgt_df_y, Elevators_source_df_y], ignore_index=True)

elevators_np_train_X = elevators_X_df.to_numpy()
elevators_np_train_y = elevators_y_df.to_numpy()

elevators_np_test_X = Elevators_tgt_df_X.to_numpy()
elevators_np_test_y = Elevators_tgt_df_y.to_numpy()

elevators_np_train_y_list = elevators_np_train_y.ravel()
elevators_np_test_y_list = elevators_np_test_y.ravel()

src_size_elevators = len(Elevators_source_df_y)
tgt_size_elevators = len(Elevators_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Target Set:  (1958, 19)
Source Set:  (6794, 19)
Test Set:  (7847, 19)
---------------------------


In [ ]:
#################################### STrAdaBoost.R2 Active Sampling Elevators ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(Elevators_tgt_df_X), len(Elevators_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_elevators = []
rmselist_stradaboost_elevators = []

for x in range(0, 50):

    model_stradaboost_elevators = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)


    y_pred_stradaboost_elevators = model_stradaboost_elevators.predict(elevators_np_test_X)

    mse_stradaboost_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_stradaboost_elevators))
    rmselist_stradaboost_elevators.append(mse_stradaboost_elevators)
        
    r2_score_stradaboost_elevators = pearsonr(elevators_np_test_y_list, y_pred_stradaboost_elevators)
    r2_score_stradaboost_elevators = (r2_score_stradaboost_elevators[0])**2
    r2scorelist_stradaboost_elevators.append(r2_score_stradaboost_elevators)


print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_elevators))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_elevators))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_elevators)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_elevators)


print("-------------------------------------------")

In [ ]:
########################### TwoStagetrAdaBoostR2 Elevators #######################################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

############# Transfer Learning specifications #########################
src_idx_elevators = np.arange(start=0, stop=(src_size_elevators - 1), step=1)
tgt_idx_elevators = np.arange(start=src_size_elevators, stop=((src_size_elevators + tgt_size_elevators)-1), step=1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}



print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_elevators = []
rmselist_TwoTrAda_elevators = []

for x in range(0, 10):

    model_TwoTrAda_elevators = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv = 10) 
    model_TwoTrAda_elevators.fit(elevators_np_train_X, elevators_np_train_y_list, src_idx_elevators, tgt_idx_elevators)

    y_pred_TwoTrAda_elevators = model_TwoTrAda_elevators.predict(elevators_np_test_X)

    mse_TwoTrAda_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_TwoTrAda_elevators))
    rmselist_TwoTrAda_elevators.append(mse_TwoTrAda_elevators)
        
    r2_score_TwoTrAda_elevators = pearsonr(elevators_np_test_y_list, y_pred_TwoTrAda_elevators)
    r2_score_TwoTrAda_elevators = (r2_score_TwoTrAda_elevators[0])**2

    r2scorelist_TwoTrAda_elevators.append(r2_score_TwoTrAda_elevators)


print("RMSE of TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_elevators))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_elevators))
print("\n")
print("RMSE of TrAdaboost.R2:", rmselist_TwoTrAda_elevators)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_elevators)


print("-------------------------------------------")

In [ ]:
######################### Gradient Boosting Regression Transfer Learning Elevators #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_elevators = []
rmselist_GBRTL_elevators = []

for x in range(0, 10):

    model_GBRTL_elevators = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)

    y_pred_GBRTL_elevators = model_GBRTL_elevators.predict(elevators_np_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_GBRTL_elevators))
    rmselist_GBRTL_elevators.append(mse_GBRTL_elevators)
        
    r2_score_GBRTL_elevators = pearsonr(elevators_np_test_y_list, y_pred_GBRTL_elevators)
    r2_score_GBRTL_elevators = (r2_score_GBRTL_elevators[0])**2
    r2scorelist_GBRTL_elevators.append(r2_score_GBRTL_elevators)


print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_elevators))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_elevators))
print("\n")
print("RMSE of GBRTL:", rmselist_GBRTL_elevators)
print("R^2 of GBRTL:", r2scorelist_GBRTL_elevators)


print("-------------------------------------------")

In [24]:
################################### Abalone ###########################################
#### range: [0.0 - 1.130]
#### Mid of correlation variable: Whole_weight
#### [0, 0.12] [0.12, 0.15], [0.15, 1.130]
#### Target variable: Rings
#######################################################################################
target_var_abalone = ['Rings']
colnames_abalone = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']
AbaloneData_df = pd.read_csv('UCI_regression/Abalone/abalone.data', header = None, names = colnames_abalone)

gender = {'M': 1,'F': 2, 'I': 3} 
AbaloneData_df.Sex = [gender[item] for item in AbaloneData_df.Sex] 

print("Abalone Data")
print(AbaloneData_df.shape)

########## Corr Abalone ################
# print("The correlation matrix is: ")
# print(AbaloneData_df.corr()['Rings'])

##################### Splitting in 3 equal parts #######################################
# print(AbaloneData_df.sort_values('Whole_weight')['Whole_weight'])
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]))

#################### Splitting with small target set and large source and test set #############
# AbaloneData_df_remain, AbaloneData_df_tgt = train_test_split(AbaloneData_df, test_size = 0.05) ## test_size = tgt size
# AbaloneData_df_source, AbaloneData_df_test = train_test_split(AbaloneData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(AbaloneData_df_tgt.shape, AbaloneData_df_source.shape, AbaloneData_df_test.shape)


drop_col_abalone = ['Whole_weight']

abalone_tgt_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]
abalone_tgt_df = abalone_tgt_df.drop(drop_col_abalone, axis = 1)
abalone_tgt_df = abalone_tgt_df.reset_index(drop=True)
print("Target Set: ",abalone_tgt_df.shape)

abalone_source1_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]
abalone_source1_df = abalone_source1_df.drop(drop_col_abalone, axis = 1)
abalone_source1_df = abalone_source1_df.reset_index(drop=True)
print("Source Set 1: ",abalone_source1_df.shape)

abalone_source2_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]
abalone_source2_df = abalone_source2_df.drop(drop_col_abalone, axis = 1)
abalone_source2_df = abalone_source2_df.reset_index(drop=True)
print("Source Set 2: ",abalone_source2_df.shape)


################################# Standardization #################################
from sklearn.preprocessing import StandardScaler
abalone_cols = abalone_tgt_df.columns.difference(['Rings'])

ss = StandardScaler()
abalone_tgt_df[abalone_cols] = ss.fit_transform(abalone_tgt_df[abalone_cols])
abalone_source1_df[abalone_cols] = ss.fit_transform(abalone_source1_df[abalone_cols])
abalone_source2_df[abalone_cols] = ss.fit_transform(abalone_source2_df[abalone_cols])


################################# Concatenating the source datasets #################################
abalone_source_df = pd.concat([abalone_source1_df, abalone_source2_df], ignore_index = True)
abalone_source_df = abalone_source_df.reset_index(drop = True)
print("Final Source Set: ",abalone_source_df.shape)


# #################### Splitting into features and target ####################
# target_abalone = ['Rings']

# abalone_source_df_y = abalone_source_df[target_abalone]
# abalone_source_df_X = abalone_source_df.drop(target_abalone, axis = 1)

# features_abalone = abalone_source_df_X.columns
# print("---------------------------")
###############################################################################################################################################

Abalone Data
(4177, 9)
Target Set:  (1241, 8)
Source Set 1:  (1465, 8)
Source Set 2:  (1471, 8)
Final Source Set:  (2936, 8)


In [25]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 60, random_state=0).fit(abalone_tgt_df)

abalone_alternate_df = abalone_tgt_df.copy()
abalone_alternate_df_np = abalone_tgt_df.to_numpy()

idxlist = []
abalone_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in abalone_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", abalone_alternate_df_np.shape)
    abalone_new_df_list.append(rowval)
    abalone_alternate_df = np.delete(abalone_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


abalone_new_df = pd.DataFrame(np.vstack(abalone_new_df_list))

print("The shape of the extracted data is: ")
print(abalone_new_df.shape)
print("-------------------------------------------------------------------")

The shape of the extracted data is: 
(60, 8)
-------------------------------------------------------------------


In [26]:
##################################################### Variance Sampling: Phase 2 ################################################

abalone_alternate_source_df = abalone_source_df.copy()
abalone_alternate_source_df_np = abalone_alternate_source_df.to_numpy()

idxlist2 = []
abalone_final_df_list = []

for row_nm in abalone_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in abalone_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", abalone_alternate_source_df_np.shape)
    abalone_final_df_list.append(row_val)
    abalone_alternate_source_df_np = np.delete(abalone_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


abalone_final_df = pd.DataFrame(np.vstack(abalone_final_df_list), columns= abalone_alternate_source_df.columns)

print("Shape of source before :",abalone_source_df.shape)
abalone_source_df = pd.DataFrame(np.vstack(abalone_alternate_source_df_np), columns= abalone_source_df.columns)
print("Shape of source after :", abalone_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", abalone_tgt_df.shape)
abalone_tgt_df = pd.concat([abalone_tgt_df, abalone_final_df], ignore_index=True)
print("Shape of target after :", abalone_tgt_df.shape)

print("----------------------------------------------")

Shape of source before : (2936, 8)
Shape of source after : (2876, 8)
----------------------------------------------
Shape of target before : (1241, 8)
Shape of target after : (1301, 8)
----------------------------------------------


In [27]:
################################## Importance Sampling: Manhattan Distance ######################################################

abalone_source_df["ManDis"] = ""

abalone_tgt_df_mean = []
prow = abalone_tgt_df.mean()
abalone_tgt_df_mean = [prow.Sex, prow.Length, prow.Diameter, prow.Height, prow.Shucked_weight,
       prow.Viscera_weight, prow.Shell_weight, prow.Rings]

rowidx = 0

for row in abalone_source_df.itertuples():
    row_list =[row.Sex, row.Length, row.Diameter, row.Height, row.Shucked_weight,
       row.Viscera_weight, row.Shell_weight, row.Rings]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = abalone_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    abalone_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

abalone_source_df = abalone_source_df.sort_values(by =['ManDis'])
abalone_source_df = abalone_source_df.head(734) 
abalone_source_df = abalone_source_df.drop(['ManDis'], axis =1)
abalone_source_df = abalone_source_df.reset_index(drop=True)

print("Target Set: ", abalone_tgt_df.shape)
print("Source Set: ", abalone_source_df.shape)

#################### Splitting into features and target ####################
target_abalone = ['Rings']

abalone_source_df_y = abalone_source_df[target_abalone]
abalone_source_df_X = abalone_source_df.drop(target_abalone, axis = 1)

features_abalone = abalone_source_df_X.columns
print("---------------------------")

###############################################################################################################################################

Target Set:  (1301, 8)
Source Set:  (600, 8)
---------------------------


In [7]:
#################################### STrAdaBoost.R2 Abalone ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_abalone = []
rmselist_stradaboost_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()
    
    sample_size = [len(abalone_source_df_X), len(abalone_train_df_X)]

    model_stradaboost_abalone = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)
    y_pred_stradaboost_abalone = model_stradaboost_abalone.predict(abalone_np_test_X)

    mse_stradaboost_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_stradaboost_abalone))
    rmselist_stradaboost_abalone.append(mse_stradaboost_abalone)
        
    r2_score_stradaboost_abalone = pearsonr(abalone_np_test_y_list, y_pred_stradaboost_abalone)
    r2_score_stradaboost_abalone = (r2_score_stradaboost_abalone[0])**2
    r2scorelist_stradaboost_abalone.append(r2_score_stradaboost_abalone)

print("RMSE List of STrAdaboostR2:", rmselist_stradaboost_abalone)
print("R^2 List of STrAdaboostR2:", r2scorelist_stradaboost_abalone)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_abalone), statistics.stdev(rmselist_stradaboost_abalone))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_abalone), statistics.stdev(r2scorelist_stradaboost_abalone))



Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboostR2: [1.281865027657627, 1.8146741902949843, 1.8009572244842176, 3.2285298785186707, 1.4732504387053216, 1.6458883209356732, 1.3570674412195836, 1.476452589118703, 1.4631428949717606, 1.214905729724244, 1.564020050989

In [6]:
########################### TwoStagetrAdaBoostR2 Abalone #######################################

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

#####################################################################################################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_abalone = []
rmselist_TwoTrAda_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()

    src_size_abalone = len(abalone_source_df_y)
    tgt_size_abalone = len(abalone_train_df_y)
    
    src_idx_abalone = np.arange(start = 0, stop = (src_size_abalone - 1), step = 1)
    tgt_idx_abalone = np.arange(start = src_size_abalone, stop = ((src_size_abalone + tgt_size_abalone)-1), step=1)


    model_TwoTrAda_abalone = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100) 
    model_TwoTrAda_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

    y_pred_TwoTrAda_abalone = model_TwoTrAda_abalone.predict(abalone_np_test_X)

    mse_TwoTrAda_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_TwoTrAda_abalone))
    rmselist_TwoTrAda_abalone.append(mse_TwoTrAda_abalone)
        
    r2_score_TwoTrAda_abalone = pearsonr(abalone_np_test_y_list, y_pred_TwoTrAda_abalone)
    r2_score_TwoTrAda_abalone = (r2_score_TwoTrAda_abalone[0])**2
    r2scorelist_TwoTrAda_abalone.append(r2_score_TwoTrAda_abalone)

print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_abalone)
print("R^2 List of TrAdaboost.R2:", r2scorelist_TwoTrAda_abalone)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_TwoTrAda_abalone), statistics.stdev(rmselist_TwoTrAda_abalone))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_TwoTrAda_abalone), statistics.stdev(r2scorelist_TwoTrAda_abalone))


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE List of TrAdaboost.R2: [2.3358154162536287, 2.6661149965277686, 2.9700755341610456, 3.2989531191802075, 2.2826498024306305, 2.0970523775620404, 2.0177380256371706, 1.8891152313622421, 2.752497388381207, 2.0542435141488022, 2.439907047558311, 2.835558420174797, 2.025763138898031, 1.9647481501725805, 2.344466033777315, 2.5560365813518255, 2.3600544608585694, 1.978096310462208, 2.646185453086457, 2.025808493314089]
R^2 List of TrAdaboost.R2: [0.49319968046465007, 0.5379373907098207, 0.46260967123422536, 0.4867972870526036, 0.6080447463524427, 0.3023703738787406, 0.42381633136776686, 0.4574925001043787, 0.3289007157474186, 0.4268819720280001, 0.5229254826839679, 0.3240890792733061, 0.6913370705969084, 0.4550277813086461, 0.4128672587159948, 0.4264490187979004, 0.5020003908303805, 0.5097270723177003, 0.4088837015973851, 0.4647667318364844]


Mean, Stdev of RMSE: 2.3770439747649466 0.3894959849588865
Mean, Stdev of R^2:

In [26]:
########################### AdaBoostR2 Transfer Learning Abalone #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTr_abalone = []
rmselist_AdaTr_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()


    model_AdaTr_abalone = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTr_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

    y_pred_AdaTr_abalone = model_AdaTr_abalone.predict(abalone_np_test_X)

    mse_AdaTr_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_AdaTr_abalone))
    rmselist_AdaTr_abalone.append(mse_AdaTr_abalone)
        
    r2_score_AdaTr_abalone = pearsonr(abalone_np_test_y_list, y_pred_AdaTr_abalone)
    r2_score_AdaTr_abalone = (r2_score_AdaTr_abalone[0])**2
    r2scorelist_AdaTr_abalone.append(r2_score_AdaTr_abalone)

print("RMSE List of Adaboost.R2 TL:", rmselist_AdaTr_abalone)
print("R^2 List of Adaboost.R2 TL:", r2scorelist_AdaTr_abalone)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_AdaTr_abalone), statistics.stdev(rmselist_AdaTr_abalone))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_AdaTr_abalone), statistics.stdev(r2scorelist_AdaTr_abalone))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2 TL: [1.2504247494019327, 1.7657249668037305, 1.89569903377933, 3.2532755632102206, 1.4936185898283525, 1.4070485067715626, 1.2780691126409347, 1.2952120111369823, 1.2185132569760504, 1.1664952928318673, 1.4431519085733768, 1.9500910194428454, 2.363560140825891, 1.1378988286910905, 1.1343452679535926, 1.511106599638056, 1.8988885837677354, 1.1572423882973184, 1.8723490916485794, 1.0998007831649441]
R^2 List of Adaboost.R2 TL: [0.5901314864496952, 0.5232221797175364, 0.5437723857569366, 0.23756770243906197, 0.6683812336475996, 0.23301984026064682, 0.6077878100647153, 0.352826224818253, 0.2830390415370179, 0.4396294459296463, 0.4683668644342236, 0.3649472972652102, 0.49633198407516443, 0.4752250341956109, 0.5698057437152155, 0.5273507252768682, 0.635721124383488, 0.5860177748898414, 0.3893848276174636, 0.4454354917910456]


Mean, Stdev of RMSE: 1.5796257847692197 0.5290940312036921
Mean, Std

In [27]:
######################### Gradient Boosting Regression Transfer Learning Abalone #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_abalone = []
rmselist_GBRTL_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()


    model_GBRTL_abalone = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

    y_pred_GBRTL_abalone = model_GBRTL_abalone.predict(abalone_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_GBRTL_abalone))
    rmselist_GBRTL_abalone.append(mse_GBRTL_abalone)
        
    r2_score_GBRTL_abalone = pearsonr(abalone_np_test_y_list, y_pred_GBRTL_abalone)
    r2_score_GBRTL_abalone = (r2_score_GBRTL_abalone[0])**2
    r2scorelist_GBRTL_abalone.append(r2_score_GBRTL_abalone)


print("RMSE List of GBRTL:", rmselist_GBRTL_abalone)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_abalone)

print("\n")


print("Mean, Stdev of RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_abalone), statistics.stdev(rmselist_GBRTL_abalone))
print("Mean, Stdev of R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_abalone), statistics.stdev(r2scorelist_GBRTL_abalone))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434

RMSE List of GBRTL: [1.396411605443678, 1.8378740199429298, 1.8271455594905333, 3.217428256994164, 1.6290472225941706, 1.2236063526759622, 1.2521234275778645, 1.373137309203999, 1.2843716646082113, 1.1118247260560632, 1.7826594807914964, 2.1438672848544416, 2.5464002094554448, 1.0332827703883227, 1.2116685836503465, 1.702678607615944, 2.0976420610431763, 1.1799921225691246, 1.8788661360109653, 1.1039443919080707]
R^2 List of GBRTL: [0.5202927821512104, 0.5122182227076909, 0.5847241682437399, 0.23466672497860805, 0.6101592506920963, 0.22992309360030042, 0.49042684746186666, 0.21661178249367366, 0.23724000592729488, 0.4362241199348688, 0.34327071306640217, 0.30017811889421864, 0.37974289243959686, 0.47304264315281974, 0.5077155240416839, 0.4240262550703565, 0.5280310657185362, 0.469579216830239, 0.3863755039169853, 0.42588065671715275]


Mean, Stdev of RMSE of GBRTL: 1.6416985896437455 0.554365305975001
Mean, Stdev of R^2 of GBRTL: 0.41551647940196706 0.12131022441344184
----------------

/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [50]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_abalone, rmselist_AdaTr_abalone)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_abalone, rmselist_GBRTL_abalone)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_abalone, rmselist_TwoTrAda_abalone)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_abalone, r2scorelist_AdaTr_abalone)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_abalone, r2scorelist_GBRTL_abalone)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_abalone, r2scorelist_TwoTrAda_abalone)
print(stats_StTr_r2, pval_StTr_r2)

1.4774158366799477 0.1778150997140501
1.4702303687021976 0.17970051016641017
0.180939186766893 0.8609152412759344


-0.5852548329307646 0.5745091033401768
-0.36585924469666253 0.7239555216280504
0.9064702983841878 0.3911716398994435


In [54]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_abalone, rmselist_AdaTr_abalone)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_abalone, rmselist_GBRTL_abalone)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_abalone, rmselist_TwoTrAda_abalone)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_abalone, r2scorelist_AdaTr_abalone)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_abalone, r2scorelist_GBRTL_abalone)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_abalone, r2scorelist_TwoTrAda_abalone)
print(stats_StTr_r2, pval_StTr_r2)

2.3950437583222772 0.07475725214402565
2.1242400051017136 0.10085945561990267
0.32488683877032 0.7615486615910663


-1.881842769362401 0.1329977130272984
-1.2603856460659317 0.2760438976285486
2.293616600098016 0.0835237480871644


In [33]:
################################################## Kinematics ######################################################################
#### range: [0.04 - 1.45]
#### Mid of correlation variable: theta7
#### [0, 0.6] [0.6, 0.85], [0.6, 0.85]
####################################################################################################################################
target_var_Kinematics = ['y']
colnames_Kinematics = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8', 'y']
KinematicsData_df = pd.read_csv('UCI_regression/Kinematics/kin8nm.data', header = None,  names = colnames_Kinematics)

print("Kinematics Data")
print(KinematicsData_df.shape)

############################################### Standardization ###############################################
kinematics_cols = KinematicsData_df.columns.difference(['y'])

ss = StandardScaler()
KinematicsData_df[kinematics_cols] = ss.fit_transform(KinematicsData_df[kinematics_cols])


########## Corr Kinematics ################
# print("The correlation matrix is: ")
# print(KinematicsData_df.corr().sort_values('y')['y'])

##################### Splitting in 3 equal parts #######################################
# print(KinematicsData_df.sort_values('theta7')['theta7'])

drop_col_kinematics = ['theta7']

kinematics_tgt_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] <= -0.5)]
kinematics_tgt_df = kinematics_tgt_df.drop(drop_col_kinematics, axis = 1)
kinematics_tgt_df = kinematics_tgt_df.reset_index(drop=True)
print("Target Set: ",kinematics_tgt_df.shape)


kinematics_source1_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > -0.5) & (KinematicsData_df['theta7'] <= 0.5)]
kinematics_source1_df = kinematics_source1_df.drop(drop_col_kinematics, axis = 1)
kinematics_source1_df = kinematics_source1_df.reset_index(drop=True)
print("Source Set 1: ",kinematics_source1_df.shape)


kinematics_source2_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > 0.5)]
kinematics_source2_df = kinematics_source2_df.drop(drop_col_kinematics, axis = 1)
kinematics_source2_df = kinematics_source2_df.reset_index(drop=True)
print("Source Set 2: ",kinematics_source2_df.shape)


################################# Standardization #################################
from sklearn.preprocessing import StandardScaler
kinematics_cols = kinematics_tgt_df.columns.difference(['y'])

ss = StandardScaler()
kinematics_tgt_df[kinematics_cols] = ss.fit_transform(kinematics_tgt_df[kinematics_cols])
kinematics_source1_df[kinematics_cols] = ss.fit_transform(kinematics_source1_df[kinematics_cols])
kinematics_source2_df[kinematics_cols] = ss.fit_transform(kinematics_source2_df[kinematics_cols])


################################# Concatenating the source datasets #################################
kinematics_source_df = pd.concat([kinematics_source1_df, kinematics_source2_df], ignore_index = True)
kinematics_source_df = kinematics_source_df.reset_index(drop = True)
print("Final Source Set: ",kinematics_source_df.shape)


# #################### Splitting into features and target (Hide in STrAdaBoost.R2)####################
# target_kinematics = ['y']

# kinematics_source_df_y = kinematics_source_df[target_kinematics]
# kinematics_source_df_X = kinematics_source_df.drop(target_kinematics, axis = 1)

# features_kinematics = kinematics_source_df_X.columns
# print("---------------------------")
###############################################################################################################################################

Kinematics Data
(8192, 9)
Target Set:  (2878, 8)
Source Set 1:  (2396, 8)
Source Set 2:  (2918, 8)
Final Source Set:  (5314, 8)


In [34]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 140, random_state=0).fit(kinematics_tgt_df)

kinematics_alternate_df = kinematics_tgt_df.copy()
kinematics_alternate_df_np = kinematics_tgt_df.to_numpy()

idxlist = []
kinematics_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in kinematics_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", kinematics_alternate_df_np.shape)
    kinematics_new_df_list.append(rowval)
    kinematics_alternate_df = np.delete(kinematics_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


kinematics_new_df = pd.DataFrame(np.vstack(kinematics_new_df_list))

print("Shape of dataset extracted: ")
print(kinematics_new_df.shape)
print("----------------------------------------------")

Shape of dataset extracted: 
(140, 8)
----------------------------------------------


In [35]:
##################################################### Variance Sampling: Phase 2 ################################################

kinematics_alternate_source_df = kinematics_source_df[1:].copy()
kinematics_alternate_source_df_np = kinematics_alternate_source_df.to_numpy()

idxlist2 = []
kinematics_final_df_list = []

for row_nm in kinematics_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in kinematics_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", Elevators_alternate_source_df_np.shape)
    kinematics_final_df_list.append(row_val)
    kinematics_alternate_source_df_np = np.delete(kinematics_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


kinematics_final_df = pd.DataFrame(np.vstack(kinematics_final_df_list), columns= kinematics_source_df.columns)

print("----------------------------------------------")
print("Shape of source before :",kinematics_source_df.shape)
kinematics_source_df = pd.DataFrame(np.vstack(kinematics_alternate_source_df_np), columns= kinematics_source_df.columns)
print("Shape of source after :", kinematics_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", kinematics_tgt_df.shape)
kinematics_tgt_df = pd.concat([kinematics_tgt_df, kinematics_final_df], ignore_index=True)
print("Shape of target after :", kinematics_tgt_df.shape)

print("----------------------------------------------")

----------------------------------------------
Shape of source before : (5314, 8)
Shape of source after : (5173, 8)
----------------------------------------------
Shape of target before : (2878, 8)
Shape of target after : (3018, 8)
----------------------------------------------


In [36]:
################################## Importance Sampling: Manhattan Distance ######################################################

kinematics_source_df["ManDis"] = ""

kinematics_tgt_df_mean = []
prow = kinematics_tgt_df.mean()
kinematics_tgt_df_mean = [prow.theta1, prow.theta2, prow.theta3, prow.theta4, prow.theta5, prow.theta6, prow.theta8, prow.y]

rowidx = 0

for row in kinematics_source_df.itertuples():
    row_list =[row.theta1, row.theta2, row.theta3, row.theta4, row.theta5, row.theta6, row.theta8, row.y]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = kinematics_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    kinematics_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

kinematics_source_df = kinematics_source_df.sort_values(by =['ManDis'])
kinematics_source_df = kinematics_source_df.head(1330) 
kinematics_source_df = kinematics_source_df.drop(['ManDis'], axis =1)
kinematics_source_df = kinematics_source_df.reset_index(drop=True)

print("Target Set: ", kinematics_tgt_df.shape)
print("Source Set: ", kinematics_source_df.shape)

#################### Splitting into features and target ####################
target_kinematics = ['y']

kinematics_source_df_y = kinematics_source_df[target_kinematics]
kinematics_source_df_X = kinematics_source_df.drop(target_kinematics, axis = 1)

features_kinematics = kinematics_source_df_X.columns
print("---------------------------")
###############################################################################################################################################

Target Set:  (3018, 8)
Source Set:  (1330, 8)
---------------------------


In [4]:
#################################### STrAdaBoost.R2 Active Learning Kinematics ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_kinematics = []
rmselist_stradaboost_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()
    
    
    sample_size = [len(kinematics_source_df_X), len(kinematics_train_df_X)]

    model_stradaboost_kinematics = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)
    y_pred_stradaboost_kinematics = model_stradaboost_kinematics.predict(kinematics_np_test_X)


    mse_stradaboost_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_stradaboost_kinematics))

    rmselist_stradaboost_kinematics.append(mse_stradaboost_kinematics)
        
    r2_score_stradaboost_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_stradaboost_kinematics)
    r2_score_stradaboost_kinematics = (r2_score_stradaboost_kinematics[0])**2

    r2scorelist_stradaboost_kinematics.append(r2_score_stradaboost_kinematics)

    
print("RMSE List of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_kinematics))
print("R^2 List of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_kinematics))

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_kinematics), statistics.stdev(rmselist_stradaboost_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_kinematics), statistics.stdev(r2scorelist_stradaboost_kinematics))


print("-------------------------------------------")

Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboostR2: 0.13609718934768414
R^2 List of STrAdaboostR2: 0.724796005642048


Mean, STDev of RMSE: 0.13609718934768414 0.008570356935774672
Mean, STDev of R^2: 0.724796005642048 0.0723719858327517
--------------------------

In [10]:
########################### TwoStagetrAdaBoostR2 Kinematics #######################################

############# Transfer Learning specifications #########################


def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_kinematics = []
rmselist_TwoTrAda_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()

    src_size_kinematics = len(kinematics_source_df_y)
    tgt_size_kinematics = len(kinematics_train_df_y)

    src_idx_kinematics = np.arange(start = 0, stop = (src_size_kinematics - 1), step = 1)
    tgt_idx_kinematics = np.arange(start = src_size_kinematics, stop = ((src_size_kinematics + tgt_size_kinematics)-1), step = 1)


    model_TwoTrAda_kinematics = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100)
    model_TwoTrAda_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

    y_pred_TwoTrAda_kinematics = model_TwoTrAda_kinematics.predict(kinematics_np_test_X)

    mse_TwoTrAda_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_TwoTrAda_kinematics))
    rmselist_TwoTrAda_kinematics.append(mse_TwoTrAda_kinematics)
        
    r2_score_TwoTrAda_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_TwoTrAda_kinematics)
    r2_score_TwoTrAda_kinematics = (r2_score_TwoTrAda_kinematics[0])**2
    r2scorelist_TwoTrAda_kinematics.append(r2_score_TwoTrAda_kinematics)

print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_kinematics)
print("R^2 List of TrAdaboostR2:", r2scorelist_TwoTrAda_kinematics)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_kinematics), statistics.stdev(rmselist_TwoTrAda_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_kinematics), statistics.stdev(r2scorelist_TwoTrAda_kinematics))


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE List of TrAdaboost.R2: [0.17127393955694623, 0.1669823459829515, 0.17217512838058419, 0.16807116672007408, 0.17662436844893892, 0.18322992269756577, 0.17708671995261255, 0.16720243538533466, 0.16972185591652802, 0.1794764312051885, 0.14985630557445656, 0.16337513788734326, 0.1650656035957635, 0.15649602793526374, 0.17227748084038497, 0.1602992884583097, 0.17564152304814457, 0.18968403888876234, 0.16677897526799754, 0.16003597223323784]
R^2 List of TrAdaboostR2: [0.5535063206087691, 0.6632203635108883, 0.5754247900707788, 0.5655199718691947, 0.5288552633967446, 0.5771643746114014, 0.44668476227349024, 0.5324714539671573, 0.5390098563208533, 0.49612229355516463, 0.6296784300489926, 0.5750741847693349, 0.36045144880565116, 0.6585029517807146, 0.4473901173060256, 0.5280390341581853, 0.5584162072512125, 0.46412984004885527, 0.6133847371223513, 0.552189676932758]


Mean, STDev of RMSE: 0.16956773339881942 0.009366466888

In [8]:
########################### AdaBoostR2 Transfer Learning Kinematics #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_kinematics = []
rmselist_AdaTL_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()

    model_AdaTL_kinematics = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

    y_pred_AdaTL_kinematics = model_AdaTL_kinematics.predict(kinematics_np_test_X)

    mse_AdaTL_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_AdaTL_kinematics))
    rmselist_AdaTL_kinematics.append(mse_AdaTL_kinematics)
        
    r2_score_AdaTL_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_AdaTL_kinematics)
    r2_score_AdaTL_kinematics = (r2_score_AdaTL_kinematics[0])**2
    r2scorelist_AdaTL_kinematics.append(r2_score_AdaTL_kinematics)

print("RMSE List of Adaboost.R2:", rmselist_AdaTL_kinematics)
print("R^2 List of Adaboost.R2:", r2scorelist_AdaTL_kinematics)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_AdaTL_kinematics), statistics.stdev(rmselist_AdaTL_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_AdaTL_kinematics), statistics.stdev(r2scorelist_AdaTL_kinematics))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [0.2154752685103698, 0.1985844838892281, 0.21146872987552065, 0.20143870658282897, 0.2084368802569673, 0.20509098740625809, 0.19590132265779048, 0.1984767295462005, 0.20066657556787076, 0.1940639339583811, 0.19573528041609659, 0.21254076626704194, 0.19260275381954683, 0.20024603801490737, 0.2127625151625833, 0.1946791338774877, 0.19844330806078467, 0.2246199493720246, 0.2070223979495842, 0.19051378455805573]
R^2 List of Adaboost.R2: [0.36537822322200547, 0.515427104077821, 0.38411375147009996, 0.40826702567999223, 0.4074368190778562, 0.5766234982070354, 0.37387393704525607, 0.3875088393617786, 0.38634178126663865, 0.48767932699211114, 0.4059204909356948, 0.3471076650608209, 0.20297093095566596, 0.4439041165518097, 0.22565914366021045, 0.34539953611115465, 0.515785387970833, 0.25395486208595136, 0.4109043013447319, 0.4318167360165548]


Mean, STDev of RMSE: 0.20293847728747644 0.008954844

In [9]:
######################### Gradient Boosting Regression Transfer Learning Kinematics #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_kinematics = []
rmselist_GBRTL_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()


    model_GBRTL_kinematics = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

    y_pred_GBRTL_kinematics = model_GBRTL_kinematics.predict(kinematics_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_GBRTL_kinematics))
    rmselist_GBRTL_kinematics.append(mse_GBRTL_kinematics)
        
    r2_score_GBRTL_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_GBRTL_kinematics)
    r2_score_GBRTL_kinematics = (r2_score_GBRTL_kinematics[0])**2

    r2scorelist_GBRTL_kinematics.append(r2_score_GBRTL_kinematics)


print("RMSE List of GBRTL:", rmselist_GBRTL_kinematics)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_kinematics)

print("\n")

    
print("Mean, STDev of RMSE:", statistics.mean(rmselist_GBRTL_kinematics), statistics.stdev(rmselist_GBRTL_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_GBRTL_kinematics), statistics.stdev(r2scorelist_GBRTL_kinematics))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434

RMSE List of GBRTL: [0.20297238222001085, 0.1848060082047616, 0.20224217250963997, 0.18303929398079735, 0.2011556891615623, 0.19600130297585558, 0.19089314700841448, 0.19118682989768113, 0.195100757148304, 0.1730455086776183, 0.18937014323012513, 0.20898403152231088, 0.19671983116526634, 0.18648861736533234, 0.2032060476394821, 0.20387651982213575, 0.18700255898624019, 0.21573602371523867, 0.19667278803892851, 0.17198487208884486]
R^2 List of GBRTL: [0.42467335025841574, 0.5648310321277084, 0.4229502742183758, 0.5042618696742892, 0.43054532168848236, 0.5429624837449435, 0.4088604238947674, 0.41530272672772856, 0.4104448645674725, 0.5500647787274874, 0.43958039663440185, 0.3666075916568952, 0.2442738102916208, 0.4949905916318364, 0.28992421680533204, 0.3281466615119518, 0.5293114951324919, 0.3242815636959829, 0.4648046415782203, 0.5053667617084855]


Mean, STDev of RMSE: 0.1940242262679275 0.011172029103735728
Mean, STDev of R^2: 0.4331092428138445 0.08960586177710513
------------------

/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [52]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_kinematics, rmselist_AdaTL_kinematics)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_kinematics, rmselist_GBRTL_kinematics)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_kinematics, rmselist_TwoTrAda_kinematics)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_kinematics, r2scorelist_AdaTL_kinematics)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_kinematics, r2scorelist_GBRTL_kinematics)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_kinematics, r2scorelist_TwoTrAda_kinematics)
print(stats_StTr_r2, pval_StTr_r2)

2.239211166142098 0.05549370834515708
5.710744979214197 0.00044878897195905153
12.933901052446585 1.2086974283329565e-06


0.15020428465586927 0.8843214418707841
-0.580453091569201 0.5775889224254931
-5.759859685898271 0.0004241772800097817


In [53]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel


stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_kinematics, rmselist_AdaTL_kinematics)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_kinematics, rmselist_GBRTL_kinematics)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_kinematics, rmselist_TwoTrAda_kinematics)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_kinematics, r2scorelist_AdaTL_kinematics)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_kinematics, r2scorelist_GBRTL_kinematics)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_kinematics, r2scorelist_TwoTrAda_kinematics)
print(stats_StTr_r2, pval_StTr_r2)

3.6296148561965667 0.02216745053260249
6.519298221944031 0.0028583835971875094
13.492049030039581 0.0001746222481559807


0.7297095707949993 0.5060026819401974
-4.064041932226505 0.015297728196215526
-9.89590741109181 0.0005852307476539434


In [46]:
################################### Computer Activity ###########################################
colnames_CompAct = ['lread', 'lwrite', 'scall', 'sread', 'swrite', 'fork', 'exec', 'rchar', 'wchar', 'pgout', 'ppgout', 
                    'pgfree', 'pgscan', 'atch', 'pgin', 'ppgin', 'pflt', 'vflt', 'runqsz', 'freemem', 'freeswap', 'usr' ]
CompActData_df = pd.read_csv('UCI_regression/ComputerActivity/cpu_act.data', header = None, names = colnames_CompAct)
print("Computer Activity Data")
print(CompActData_df.shape)

##################### Corr Computer Activity #####################
# print("The correlation matrix is: ")
# print(CompActData_df.corr().sort_values('usr')['usr'])

##################### Splitting in 3 equal parts #######################################
# print(CompActData_df.sort_values('pgin')['pgin'])

drop_col_compact = ['pgin']

compact_tgt_df = CompActData_df.loc[(CompActData_df['pgin'] <= 1.0)]
compact_tgt_df = compact_tgt_df.drop(drop_col_compact, axis = 1)
compact_tgt_df = compact_tgt_df.reset_index(drop=True)
print("Target Set: ",compact_tgt_df.shape)


compact_source1_df = CompActData_df.loc[(CompActData_df['pgin'] > 1.0) & (CompActData_df['pgin'] <= 6.0)]
compact_source1_df = compact_source1_df.drop(drop_col_compact, axis = 1)
compact_source1_df = compact_source1_df.reset_index(drop = True)
print("Source Set 1: ",compact_source1_df.shape)


compact_source2_df = CompActData_df.loc[(CompActData_df['pgin'] > 6.0)]
compact_source2_df = compact_source2_df.drop(drop_col_compact, axis = 1)
compact_source2_df = compact_source2_df.reset_index(drop = True)
print("Source Set 2: ",compact_source2_df.shape)


################################# Standardization #################################
from sklearn.preprocessing import StandardScaler
compact_cols = compact_tgt_df.columns.difference(['usr'])

ss = StandardScaler()
compact_tgt_df[compact_cols] = ss.fit_transform(compact_tgt_df[compact_cols])
compact_source1_df[compact_cols] = ss.fit_transform(compact_source1_df[compact_cols])
compact_source2_df[compact_cols] = ss.fit_transform(compact_source2_df[compact_cols])


##################### Concatenating the source datasets #####################
compact_source_df = pd.concat([compact_source1_df, compact_source2_df], ignore_index = True)
compact_source_df = compact_source_df.reset_index(drop = True)
print("Final Source Set: ",compact_source_df.shape)


# #################### Splitting into features and target (Hide for STrAdaBoost.R2)####################
# target_compact = ['usr']

# compact_source_df_y = compact_source_df[target_compact]
# compact_source_df_X = compact_source_df.drop(target_compact, axis = 1)

# features_compact = compact_source_df_X.columns
# print("---------------------------")
###############################################################################################################################################

Computer Activity Data
(8192, 22)
Target Set:  (2766, 21)
Source Set 1:  (2539, 21)
Source Set 2:  (2887, 21)
Final Source Set:  (5426, 21)


In [47]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 200, random_state=0).fit(compact_tgt_df)

compact_alternate_df = compact_tgt_df.copy()
compact_alternate_df_np = compact_tgt_df.to_numpy()

idxlist = []
compact_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in compact_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", compact_alternate_df_np.shape)
    compact_new_df_list.append(rowval)
    compact_alternate_df = np.delete(compact_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


compact_new_df = pd.DataFrame(np.vstack(compact_new_df_list))

print("The dataset extracted is: ")
print(compact_new_df.shape)
print("------------------------------------------------------")

##################################################### Variance Sampling: Phase 2 ################################################

compact_alternate_source_df = compact_source_df.copy()
compact_alternate_source_df_np = compact_alternate_source_df.to_numpy()

idxlist2 = []
compact_final_df_list = []

for row_nm in compact_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in compact_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", compact_alternate_source_df_np.shape)
    compact_final_df_list.append(row_val)
    compact_alternate_source_df_np = np.delete(compact_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


compact_final_df = pd.DataFrame(np.vstack(compact_final_df_list), columns= compact_alternate_source_df.columns)

print("Shape of source before :",compact_source_df.shape)
compact_source_df = pd.DataFrame(np.vstack(compact_alternate_source_df_np), columns= compact_source_df.columns)
print("Shape of source after :", compact_source_df.shape)

print("----------------------------------------------")

print("Shape of target before :", compact_tgt_df.shape)
Elevators_tgt_df = pd.concat([compact_tgt_df, compact_final_df], ignore_index=True)
print("Shape of target after :", compact_tgt_df.shape)

print("----------------------------------------------")

The dataset extracted is: 
(200, 21)
------------------------------------------------------
Shape of source before : (5426, 21)
Shape of source after : (5226, 21)
----------------------------------------------
Shape of target before : (2766, 21)
Shape of target after : (2766, 21)
----------------------------------------------


In [48]:
################################## Importance Sampling: Manhattan Distance ######################################################
compact_source_df["ManDis"] = ""

compact_tgt_df_mean = []
prow = compact_tgt_df.mean()
compact_tgt_df_mean = [prow.lread, prow.lwrite, prow.scall, prow.sread, prow.swrite, prow.fork, prow.exec, prow.rchar,
       prow.wchar, prow.pgout, prow.ppgout, prow.pgfree, prow.pgscan, prow.atch, prow.ppgin, prow.pflt, prow.vflt, prow.runqsz,
        prow.freemem, prow.freeswap, prow.usr]

rowidx = 0

for row in compact_source_df.itertuples():
    row_list =[row.lread, row.lwrite, row.scall, row.sread, row.swrite, row.fork, row.exec, row.rchar,
       row.wchar, row.pgout, row.ppgout, row.pgfree, row.pgscan, row.atch, row.ppgin, row.pflt, row.vflt, row.runqsz,
        row.freemem, row.freeswap, row.usr]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = compact_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    compact_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1


# kinematics_source_df = kinematics_source_df.sort_values(by =['ManDis'])
# kinematics_source_df = kinematics_source_df.head(1330) 
# kinematics_source_df = kinematics_source_df.drop(['ManDis'], axis =1)
# kinematics_source_df = kinematics_source_df.reset_index(drop=True)

# print("Target Set: ", kinematics_tgt_df.shape)
# print("Source Set: ", kinematics_source_df.shape)

compact_source_df = compact_source_df.sort_values(by =['ManDis'])
compact_source_df = compact_source_df.head(1356) #### For Computer Activity 650 instances
compact_source_df = compact_source_df.drop(['ManDis'], axis =1)
compact_source_df = compact_source_df.reset_index(drop=True)

print("Target Set: ", compact_tgt_df.shape)
print("Source Set: ", compact_source_df.shape)


#################### Splitting into features and target ####################
target_compact = ['usr']

compact_source_df_y = compact_source_df[target_compact]
compact_source_df_X = compact_source_df.drop(target_compact, axis = 1)

features_compact = compact_source_df_X.columns
print("---------------------------")
###############################################################################################################################################

Target Set:  (2766, 21)
Source Set:  (1356, 21)
---------------------------


In [10]:
#################################### STrAdaBoost.R2 compAct ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_compact = []
rmselist_stradaboost_compact = []

print("STrAdaboost.R2 (50 iter)")
print("-------------------------------------------")

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()
    
    sample_size = [len(compact_source_df_X), len(compact_train_df_X)]

    model_stradaboost_compact = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_compact.fit(compact_np_train_X, compact_np_train_y_list)
    y_pred_stradaboost_compact = model_stradaboost_compact.predict(compact_np_test_X)
    
    mse_stradaboost_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_stradaboost_compact))
    rmselist_stradaboost_compact.append(mse_stradaboost_compact)
        
    r2_score_stradaboost_compact = pearsonr(compact_np_test_y_list, y_pred_stradaboost_compact)
    r2_score_stradaboost_compact = (r2_score_stradaboost_compact[0])**2
    r2scorelist_stradaboost_compact.append(r2_score_stradaboost_compact)
        

print("RMSE List of STrAdaboost.R2:", rmselist_stradaboost_compact)
print("R^2 List of STrAdaboost.R2:", r2scorelist_stradaboost_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_compact), statistics.stdev(rmselist_stradaboost_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_compact), statistics.stdev(r2scorelist_stradaboost_compact))



STrAdaboost.R2 (50 iter)
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboost.R2: [2.720968733669425, 2.4345699191658334, 2.541768848715657, 2.2897283428819843, 2.6311561694449166, 2.5474782829011366, 2.3713362266180478, 2.2

In [15]:
########################### TwoStagetrAdaBoostR2 compAct #######################################

############# Transfer Learning specifications #########################

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

#####################################################################################################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_compact = []
rmselist_TwoTrAda_compact = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()

    src_size_compact = len(compact_source_df_y)
    tgt_size_compact = len(compact_train_df_y)
    
    src_idx_compact = np.arange(start=0, stop=(src_size_compact - 1), step=1)
    tgt_idx_compact = np.arange(start=src_size_compact, stop=((src_size_compact + tgt_size_compact)-1), step=1)

    model_TwoTrAda_compact = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100)
    model_TwoTrAda_compact.fit(compact_np_train_X, compact_np_train_y_list, src_idx_compact, tgt_idx_compact)

    y_pred_TwoTrAda_compact = model_TwoTrAda_compact.predict(compact_np_test_X)

    mse_TwoTrAda_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_TwoTrAda_compact))

    rmselist_TwoTrAda_compact.append(mse_TwoTrAda_compact)
        
    r2_score_TwoTrAda_compact = pearsonr(compact_np_test_y_list, y_pred_TwoTrAda_compact)
    r2_score_TwoTrAda_compact = (r2_score_TwoTrAda_compact[0])**2

    r2scorelist_TwoTrAda_compact.append(r2_score_TwoTrAda_compact)


print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_compact)
print("R^2 List of TrAdaboost.R2:", r2scorelist_TwoTrAda_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_compact), statistics.stdev(rmselist_TwoTrAda_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_compact), statistics.stdev(r2scorelist_TwoTrAda_compact))


print("-------------------------------------------")

RMSE List of TrAdaboost.R2: [2.850245727000064, 2.7005677832721795, 2.423316286110016, 2.3887951407597274, 2.4424424368999618, 2.712540816344094, 2.6922048694063783, 2.4355915511440815, 2.273891051582728, 2.455530672645627, 2.791109298242016, 3.0911190772016943, 2.452726097045739, 2.4119787667080628, 2.7179839042457594, 2.7054580881290557, 2.6361848259425584, 2.715932861277004, 2.646284569087001, 2.320419939336392]
R^2 List of TrAdaboost.R2: [0.9558602992700392, 0.931005062205565, 0.9444198699483178, 0.9767082601341441, 0.9821184016071886, 0.9787744231494595, 0.8357842797316652, 0.9680639432406996, 0.9724770595594833, 0.9748248196263483, 0.8432705081475351, 0.9211716882582086, 0.9661358344788633, 0.9474209043007024, 0.7855117137776909, 0.9814564031631141, 0.9710649039181786, 0.8450924978885074, 0.9341749920943405, 0.9798138826886302]


Mean, STDev of RMSE: 2.593216188119007 0.20554703753186954
Mean, STDev of R^2: 0.9347574873594341 0.058924264017501
------------------------------------

In [16]:
########################### AdaBoostR2 Transfer Learning compAct #####################################################
from sklearn.ensemble import AdaBoostRegressor
from keras.callbacks import EarlyStopping

#earlystopping = EarlyStopping(monitor="mean_squared_error", patience=40, verbose=1, mode='auto')
#epochs = 100, batch_size = 5, callbacks=[earlystopping])

print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_compact = []
rmselist_AdaTL_compact = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()

    model_AdaTL_compact = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_compact.fit(compact_np_train_X, compact_np_train_y_list) 

    y_pred_AdaTL_compact = model_AdaTL_compact.predict(compact_np_test_X)

    mse_AdaTL_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_AdaTL_compact))
    rmselist_AdaTL_compact.append(mse_AdaTL_compact)
        
    r2_score_AdaTL_compact = pearsonr(compact_np_test_y_list, y_pred_AdaTL_compact)
    r2_score_AdaTL_compact = (r2_score_AdaTL_compact[0])**2
    r2scorelist_AdaTL_compact.append(r2_score_AdaTL_compact)


print("RMSE List of Adaboost.R2:", rmselist_AdaTL_compact)
print("R^2 List of Adaboost.R2:", r2scorelist_AdaTL_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_AdaTL_compact), statistics.stdev(rmselist_AdaTL_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_AdaTL_compact), statistics.stdev(r2scorelist_AdaTL_compact))

print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [2.7497607287843904, 2.5208526259990593, 2.4948268134683627, 2.577512548409647, 2.32702382173679, 2.3216455841878125, 2.4181591995786937, 2.334494866576656, 2.3519531779585354, 2.60371142986263, 2.631725106318242, 2.8575501136246153, 2.54642011764177, 2.305717861419587, 2.722873833819519, 2.6456650763593865, 2.6419954948055775, 2.7446544956999404, 2.7618707632104353, 2.6573848223593988]
R^2 List of Adaboost.R2: [0.9628507856331731, 0.9476199758914179, 0.9444863522653001, 0.9739300893739561, 0.9862879199834731, 0.9853015927797946, 0.8881935735983749, 0.9726899441917263, 0.9752572746248802, 0.9735256726643179, 0.8733316440291593, 0.9347874453456327, 0.9669435715720194, 0.9524811342596715, 0.8268690783935556, 0.9833335382643411, 0.9725179154073612, 0.8344019712614956, 0.9428754162773287, 0.9786248389406393]


Mean, STDev of RMSE: 2.5607899240910523 0.17056972694345388
Mean, STDev of R^2: 0.

In [ ]:
######################### Gradient Boosting Regression Transfer Learning compAct #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_compact = []
rmselist_GBRTL_compact = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()


    model_GBRTL_compact = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100) #, subsample=0.5)
    model_GBRTL_compact.fit(compact_np_train_X, compact_np_train_y_list)

    y_pred_GBRTL_compact = model_GBRTL_compact.predict(compact_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_GBRTL_compact))
    rmselist_GBRTL_compact.append(mse_GBRTL_compact)
        
    r2_score_GBRTL_compact = pearsonr(compact_np_test_y_list, y_pred_GBRTL_compact)
    r2_score_GBRTL_compact = (r2_score_GBRTL_compact[0])**2
    r2scorelist_GBRTL_compact.append(r2_score_GBRTL_compact)


print("RMSE List of GBRTL:", rmselist_GBRTL_compact)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_GBRTL_compact), statistics.stdev(rmselist_GBRTL_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_GBRTL_compact), statistics.stdev(r2scorelist_GBRTL_compact))

print("-------------------------------------------")